# BLIP (Bootstrapped Language-Image Pretraining)

In [4]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base", token=HF_TOKEN)
model = BlipForConditionalGeneration.from_pretrained('Salesforce/blip-image-captioning-base', token=HF_TOKEN)

image = Image.open('test.jpg').convert('RGB')
inputs = processor(images=image, return_tensors='pt')

output = model.generate(**inputs)

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [5]:
print("BLIP caption:", processor.decode(output[0], skip_special_tokens=True))

BLIP caption: a puppy sitting in the grass with its mouth open


# CLIP (Contrastive Language-Image Pretraining)

In [6]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-xwddk2ia
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-xwddk2ia
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

model, processor = clip.load("ViT-B/32", device=device)

image = processor(Image.open('test.jpg')).unsqueeze(0).to(device)

caption_options = [
    "a dog on the grass",
    "a pug sitting",
    "a cute cat",
    "a cat on the table"
]

captions = clip.tokenize(caption_options).to(device)

with torch.no_grad():
  image_features = model.encode_image(image)
  text_features = model.encode_text(captions)
  logits_per_image, _ = model(image, captions)
  probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("CLIP's best caption:", caption_options[probs.argmax()])

CLIP's best caption: a dog on the grass


# [실습] 이미지 캡션
1. BLIP을 이용해 이미지에 적절한 캡션 생성
2. OpenAI API를 이용해 보기가 될 캡션 생성
3. 사용자의 선택
4. CLIP을 이용해 가장 유사도가 높은 캡션 매칭
5. 결과 출력: 정답/오답 여부, BLIP 생성 캡션, CLIP 매칭 캡션, CLIP 유사도 점수